### Explanation of Maintenance Routines:
- **Vacuum**: Removes old files to free up space.
- **Optimize**: Combines small files to improve read performance.
- **Z-Ordering**: Organizes data to improve performance in filtered queries.
- **History/Time Travel**: Audits and accesses previous versions of data.

### Explicação das Rotinas de Manutenção:
- **Vacuum**: Remove arquivos antigos para liberar espaço.
- **Optimize**: Combina arquivos pequenos para melhorar a performance de leitura.
- **Z-Ordering**: Organiza os dados para melhorar o desempenho em consultas filtradas.
- **History/Time Travel**: Audita e acessa versões anteriores dos dados.



Maintaining a well-managed Delta Lake is essential to ensuring performance, data integrity, and efficient use of resources. Here are the main Delta Lake maintenance routines, when, how, and why to use them:

### 1. Vacuum
**When to use**: To remove old files that are no longer needed, such as those generated by update, merge, or delete operations.

**Why to use**: Delta Lake maintains old versions of data (history) to provide features such as time travel and rollback. Over time, these old files can consume a lot of disk space. Vacuum removes these files, freeing up space.

**Recommendation**: Avoid setting the retention period below 7 days without considering the implications for time travel. The default of 7 days is safe to maintain data recoverability while cleaning up obsolete files.


Manter um** Delta Lake** bem gerenciado é fundamental para garantir a performance, a integridade dos dados e o uso eficiente de recursos. Aqui estão as principais rotinas de manutenção do Delta Lake, quando, como e por que usá-las:

### 1. Vacuum
**Quando usar**: Para remover arquivos antigos que não são mais necessários, como aqueles gerados por operações de update, merge ou delete.

**Por que usar**: O Delta Lake mantém versões antigas de dados (histórico) para fornecer recursos como time travel e rollback. Com o tempo, esses arquivos antigos podem consumir muito espaço em disco. O vacuum remove esses arquivos, liberando espaço.

**Recomendação**: Evite configurar o período de retenção abaixo de 7 dias sem considerar as implicações no time travel. O padrão de 7 dias é seguro para manter a possibilidade de recuperação de dados e, ao mesmo tempo, limpar arquivos obsoletos. (PT-BR)

In [0]:
from delta.tables import DeltaTable

gold_path = "/mnt/lhdw/layer_gold/fato_acidentes/"

# Disable retention duration check
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

# Run vacuum to remove unused files older than 7 days
# Executa vacuum para remover arquivos não utilizados com mais de 7 dias (PT-BR)
delta_table = DeltaTable.forPath(spark, gold_path)
delta_table.vacuum(7)


Out[1]: DataFrame[]

### 2. (Optimize)
How to use ?: To optimize the layout of files stored in Delta Lake, especially after many write or update operations that can result in small files.

Why use ?: Delta Lake can end up with many small files after write or merge operations. This can hurt query performance due to the overhead of reading many files. Optimize combines small files into larger files, improving read and processing.

Recommendation: Use optimize at regular intervals or after large write operations to ensure that the data layout remains efficient. To further improve performance, optimize can be combined with Z-Ordering.

### 2. (Optimize)
como usar ?: Para otimizar o layout dos arquivos armazenados no Delta Lake, especialmente após muitas operações de escrita ou atualização que podem gerar arquivos pequenos.

Por que usar ?: O Delta Lake pode acabar com muitos arquivos pequenos após operações de escrita ou merge. Isso pode prejudicar o desempenho das consultas devido ao overhead de leitura de muitos arquivos. O optimize combina arquivos pequenos em arquivos maiores, melhorando a leitura e o processamento.

Recomendação: Use o optimize em intervalos regulares ou após grandes operações de escrita, para garantir que o layout dos dados continue eficiente. Para melhorar ainda mais o desempenho, o optimize pode ser combinado com Z-Ordering. (PT-BR)

In [0]:
from delta.tables import *
# Define os caminhos de armazenamento no Data Lake
gold_path = "/mnt/lhdw/layer_gold/fato_acidentes/"
# Otimiza a tabela combinando arquivos pequenos em arquivos maiores
delta_table = DeltaTable.forPath(spark, gold_path)
delta_table.optimize().executeCompaction()


Out[2]: DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bi


## 3. Z-Ordering
When to use it?: To optimize queries that frequently filter on specific columns, such as date or key columns.

Why to use it?: Z-Ordering improves read performance by physically organizing data on disk based on a column or set of columns, reducing the time required to fetch the filtered records.

Recommended?: Use Z-Ordering on columns that are frequently used in filter clauses to improve readability of related data. Combine this with optimize to have more efficiently organized data on disk.


### 3. Z-Ordering
Quando usar ?:Para otimizar as consultas que realizam filtragens frequentes em determinadas colunas, como colunas de data ou chave.

Por que usar?: O Z-Ordering melhora o desempenho da leitura ao organizar fisicamente os dados em disco com base em uma coluna ou conjunto de colunas, reduzindo o tempo necessário para buscar os registros filtrados.

Recomendação?: Use o Z-Ordering em colunas que são frequentemente usadas em cláusulas de filtro para melhorar a leitura de dados relacionados. Combine isso com o optimize para ter dados organizados de forma mais eficiente no disco. (PT-BR)

In [0]:
from delta.tables import DeltaTable
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


gold_path = "/mnt/lhdw/layer_gold/fato_acidentes/"

delta_table = DeltaTable.forPath(spark, gold_path)

def vacuum_table(retention_hours=168):
    """
    Remove arquivos antigos (default: 168 horas = 7 dias).
    """
    print(f"Executando VACUUM com retenção de {retention_hours} horas...")
    delta_table.vacuum(retention_hours)
    print("Vacuum concluído.")

def optimize_table_zorder():
    """
    Otimiza a tabela aplicando Z-Ordering pela coluna SK_tempo.
    """
    print("Executando OPTIMIZE com Z-Ordering pela coluna SK_tempo...")
    spark.sql(f"""OPTIMIZE delta.`{gold_path}` ZORDER BY (SK_tempo)""")
    print("Optimize com Z-Ordering concluído.")

def show_history():
    """
    Exibe o histórico da tabela Delta (Time Travel).
    """
    print("Histórico de transações:")
    history_df = delta_table.history()
    display(history_df)

def update_records(filter_condition, update_expr):
    """
    Atualiza registros conforme condição.
    Exemplo: update_records("Ano = 2020", {"mortos": "0"})
    """
    print(f"Atualizando registros onde {filter_condition}...")
    delta_table.update(
        condition=filter_condition,
        set=update_expr
    )
    print("Update concluído.")

def delete_records(filter_condition):
    """
    Remove registros conforme condição.
    Exemplo: delete_records("ignorados > 0")
    """
    print(f"Deletando registros onde {filter_condition}...")
    delta_table.delete(filter_condition)
    print(" Delete concluído.")



### 5. History and Time Travel
When to use it? : To audit changes to the Delta table or to access previous versions of the data.

Why to use it? : Delta Lake maintains a transaction log that allows you to track all modifications made to the table. This is useful for auditing and recovering data to a previous point in time.

Recommended? : Use history and time travel to debug issues or restore previous versions of the data when necessary. However, remember to use vacuum to manage the amount of history kept.

### 5. History e Time Travel
Quando usar? : Para auditar mudanças na tabela Delta ou para acessar versões anteriores dos dados.

Por que usar ? : O Delta Lake mantém um log de transações que permite rastrear todas as modificações feitas na tabela. Isso é útil para auditoria e recuperação de dados em um ponto anterior no tempo.

Recomendação? : Use o histórico e o time travel para depurar problemas ou restaurar versões anteriores dos dados quando necessário. No entanto, lembre-se de usar o vacuum para gerenciar a quantidade de histórico mantido. (PT-BR)

### Accessing table history:

### Acessando o histórico da tabela: (PT-BR)

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import *

# Visualizar o histórico da tabela Delta
df_historico = DeltaTable.forPath(spark, "/mnt/lhdw/layer_gold/fato_acidentes").history()

display(df_historico)

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
19,2025-05-21T02:19:18.000+0000,3991693015469206,clay_cloud@hotmail.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [], batchId -> 0, auto -> false)",null,List(1870597478109446),0521-005749-9mxgmz85,18,SnapshotIsolation,false,"Map(numRemovedFiles -> 6, numRemovedBytes -> 485315, p25FileSize -> 464457, numDeletionVectorsRemoved -> 0, minFileSize -> 464457, numAddedFiles -> 1, maxFileSize -> 464457, p75FileSize -> 464457, p50FileSize -> 464457, numAddedBytes -> 464457)",null,Databricks-Runtime/12.2.x-scala2.12
18,2025-05-21T01:44:47.000+0000,3991693015469206,clay_cloud@hotmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4338771034993245),0521-005749-9mxgmz85,17,WriteSerializable,false,"Map(numFiles -> 6, numOutputRows -> 16715, numOutputBytes -> 485315)",null,Databricks-Runtime/12.2.x-scala2.12
17,2025-05-19T13:52:58.000+0000,3991693015469206,clay_cloud@hotmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4338771034993245),0519-132418-r7rmca2x,16,WriteSerializable,false,"Map(numFiles -> 6, numOutputRows -> 16715, numOutputBytes -> 485631)",null,Databricks-Runtime/12.2.x-scala2.12
16,2025-05-19T11:33:04.000+0000,3991693015469206,clay_cloud@hotmail.com,RESTORE,"Map(version -> 2, timestamp -> null)",null,List(1870597478109446),0519-110908-k93zsuug,15,Serializable,false,"Map(numRestoredFiles -> 6, removedFilesSize -> 462795, numRemovedFiles -> 1, restoredFilesSize -> 485550, numOfFilesAfterRestore -> 6, tableSizeAfterRestore -> 485550)",null,Databricks-Runtime/12.2.x-scala2.12
15,2025-05-19T11:32:13.000+0000,3991693015469206,clay_cloud@hotmail.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [], batchId -> 0, auto -> false)",null,List(1870597478109446),0519-110908-k93zsuug,14,SnapshotIsolation,false,"Map(numRemovedFiles -> 6, numRemovedBytes -> 485854, p25FileSize -> 462795, numDeletionVectorsRemoved -> 0, minFileSize -> 462795, numAddedFiles -> 1, maxFileSize -> 462795, p75FileSize -> 462795, p50FileSize -> 462795, numAddedBytes -> 462795)",null,Databricks-Runtime/12.2.x-scala2.12
14,2025-05-19T11:20:30.000+0000,3991693015469206,clay_cloud@hotmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4338771034993245),0519-110908-k93zsuug,13,WriteSerializable,false,"Map(numFiles -> 6, numOutputRows -> 16715, numOutputBytes -> 485854)",null,Databricks-Runtime/12.2.x-scala2.12
13,2025-05-18T15:14:29.000+0000,3991693015469206,clay_cloud@hotmail.com,RESTORE,"Map(version -> 2, timestamp -> null)",null,List(1870597478109446),0518-141425-3w2ti7p4,12,Serializable,false,"Map(numRestoredFiles -> 6, removedFilesSize -> 463120, numRemovedFiles -> 1, restoredFilesSize -> 485550, numOfFilesAfterRestore -> 6, tableSizeAfterRestore -> 485550)",null,Databricks-Runtime/12.2.x-scala2.12
12,2025-05-18T15:13:39.000+0000,3991693015469206,clay_cloud@hotmail.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [], batchId -> 0, auto -> false)",null,List(1870597478109446),0518-141425-3w2ti7p4,11,SnapshotIsolation,false,"Map(numRemovedFiles -> 6, numRemovedBytes -> 484981, p25FileSize -> 463120, numDeletionVectorsRemoved -> 0, minFileSize -> 463120, numAddedFiles -> 1, maxFileSize -> 463120, p75FileSize -> 463120, p50FileSize -> 463120, numAddedBytes -> 463120)",null,Databricks-Runtime/12.2.x-scala2.12
11,2025-05-18T15:08:36.000+0000,3991693015469206,clay_cloud@hotmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(4338771034993245),0518-141425-3w2ti7p4,10,WriteSerializable,false,"Map(numFiles -> 6, numOutputRows -> 16715, numOutputBytes -> 484981)",null,Databricks-Runtime/12.2.x-scala2.12
10,2025-05-18T14:50:21.000+0000,3991693015469206,clay_cloud@hotmail.com,RESTORE,"Map(version -> 2, timestamp -> null)",null,List(1870597478109446),0518-141425-3w2ti7p4,9,Serializable,false,"Map(numRestoredFiles -> 6, removedFilesSize -> 463938, numRe

### Access old versions (Time Travel):

### Acessar versões antigas (Time Travel): (PT-BR)

In [0]:
# Acessar a versão 5 da tabela
df = spark.read.format("delta").option("versionAsOf", 2).load("/mnt/lhdw/layer_gold/fato_acidentes")
display(df)


org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 489.0 failed 1 times, most recent failure: Lost task 0.0 in stage 489.0 (TID 1900) (ip-10-172-161-226.us-west-2.compute.internal executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/mnt/lhdw/layer_gold/fato_acidentes/part-00001-1d4e4005-7f5f-4b55-ab65-0374803435cd-c000.snappy.parquet.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:704)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:673)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$prepareNextFile$1(FileScanRDD.scala:879)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.$anonfun$run$1(SparkThreadLocalForwardingThreadPoolExecutor.scala:116)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.IdentityClaim$.withClaim(IdentityClaim.scala:48)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.$anonfun$runWithCaptured$4(SparkThreadLocalForwardingThreadPoolExecutor.scala:79)
	at com.databricks.unity.UCSEphemeralState$Handle.runWith(UCSEphemeralState.scala:41)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:78)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.runWithCaptured$(SparkThreadLocalForwardingThreadPoolExecutor.scala:64)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:113)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.run(SparkThreadLocalForwardingThreadPoolExecutor.scala:116)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
Caused by: java.util.concurrent.ExecutionException: java.io.FileNotFoundException: /mnt/lhdw/layer_gold/fato_acidentes/part-00001-1d4e4005-7f5f-4b55-ab65-0374803435cd-c000.snappy.parquet
	at java.util.concurrent.CompletableFuture.reportGet(CompletableFuture.java:357)
	at java.util.concurrent.CompletableFuture.get(CompletableFuture.java:1908)
	at com.databricks.spark.metrics.FileSystemWithMetrics.openFileWithOptions(FileSystemWithMetrics.scala:363)
	at org.apache.hadoop.fs.FileSystem$FSDataInputStreamBuilder.build(FileSystem.java:4768)
	at com.databricks.spark.util.DatabricksRangeFetcher.fetchRange(DatabricksRangeFetcher.scala:52)
	at com.databricks.io.RangeFetcher$.fetch(RangeFetcher.scala:79)
	at com.databricks.io.RangeFetcher.fetch(RangeFetcher.scala)
	at com.databricks.sql.io.parquet.CachingParquetFileReader$FooterByteReader.readTail(CachingParquetFileReader.java:362)
	at com.databricks.sql.io.parquet.CachingParquetFileReader$FooterByteReader.read(CachingParquetFileReader.java:381)
	at com.databricks.sql.io.parquet.CachingParquetFooterReader$UnifiedCache.read(CachingParquetFooterReader.java:417)
	at com.databricks.sql.io.parquet.CachingParquetFooterReader.lambda$readFooterFromStorage$0(CachingParquetFooterReader.java:284)
	at org.apache.spark.util.JavaFrameProfiler.record(JavaFrameProfiler.java:17)
	at com.databricks.sql.io.parquet.CachingParquetFooterReader.readFooterFromStorage(CachingParquetFooterReader.java:280)
	at com.databricks.sql.io.parquet.Ca

### 6. Restore an old version of Delta table
### 6. Restaurar uma versão antiga de tabela Delta (PT_BR)

In [0]:
delta_table_path = "dbfs:/mnt/lhdw/layer_gold/fato_acidentes"

# Verifica se é uma tabela Delta válida antes de continuar
display(dbutils.fs.ls(f"{delta_table_path}/_delta_log"))

# Registrar tabela temporária e restaurar
spark.sql("DROP TABLE IF EXISTS temp_restore")
spark.sql(f"CREATE TABLE temp_restore USING DELTA LOCATION '{delta_table_path}'")
spark.sql("RESTORE TABLE temp_restore TO VERSION AS OF 2")

# Exibir dados restaurados
display(spark.read.format("delta").load(delta_table_path))
